In [16]:

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
from scipy.linalg import expm
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_step_heisenberg_qt, ham_evol_qt


### State we should get

$$\rho' = \mathbb{I} + \delta \left[ \sum_\omega \gamma(\omega) \left(-\frac{1}{2} \{\sigma_x(\omega)^\dagger \sigma_x(\omega), \rho\} + \sigma_x(\omega) \rho \sigma_x(\omega)^\dagger\right) \right]+ \mathcal{O}(\delta^2)$$
Where we used the $\sigma_x$ jump on the 1st system qubit out of the [0, 1, 2]. Here it is still under inverstigation how large should $\delta$ be s.t we can really drop the higher orders, and also that we only have those terms there that should be dropped.

In [17]:
num_qubits = 3
num_energy_bits = 6
delta = 0.1 
eps = 0.1
sigma = 5
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1
T = 1
total_time = T
num_trotter_steps = 100

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)

step_size = total_time / num_trotter_steps
trott_step_qt = trotter_step_heisenberg_qt(num_qubits, step_size, coeffs=hamiltonian.rescaled_coeffs)
trott_U_qt = ham_evol_qt(num_qubits, trott_step_qt, num_trotter_steps)

# U = expm(1j * total_time * hamiltonian.qt.full())
# print(np.linalg.norm(U - trott_U_qt.full()))

#* Initial state
with open(f'/Users/bence/code/liouvillian_metro/data/initial_state_n{num_qubits}k{num_energy_bits}d{delta}.pkl', 'rb') as f:
    initial_state = pickle.load(f)

initial_dm = DensityMatrix(initial_state).data

Original spectrum:  [-3.7    -3.177  -2.5482 -1.7     1.625   2.4547  3.2521  3.7936]
Ideal spectrum:  [0.     0.0314 0.0692 0.1201 0.3198 0.3696 0.4175 0.45  ]
Nonrescaled coefficients:  [1.  0.8 0.9 1. ]
Rescaled coefficients:  [0.06005156 0.04804125 0.05404641 0.06005156]


In [18]:
N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
energy_labels = 2 * np.pi * N_labels / N 
time_labels = N_labels

jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()

boltzmann = lambda beta, energy: np.min([1, np.exp(-beta * energy)])

#* Exact
dm_we_should_get = initial_dm
for phase in N_labels:
    energy = 2 * np.pi * phase / 2**num_energy_bits
    oft_op = oft(jump_op, phase, 2**num_energy_bits, sigma, hamiltonian=hamiltonian.qt.full())
    dm_we_should_get += delta * boltzmann(beta, energy)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
                                      -0.5 * initial_dm @ oft_op.conj().T @ oft_op
                                      + oft_op @ initial_dm @ oft_op.conj().T)

#* Trotter
# for w in energy_labels:
#     oft_op = oft(jump_op, w, 2**num_energy_bits, sigma, trotter=trott_U_qt.full())
#     dm_we_should_get += delta * boltzmann(beta, w)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
#                                                     -0.5 * initial_dm @ oft_op.conj().T @ oft_op
#                                                     + oft_op @ initial_dm @ oft_op.conj().T)
    
dm_we_should_get /= np.trace(dm_we_should_get)


In [19]:
with open(f'/Users/bence/code/liouvillian_metro/data/state_before_measure_n{num_qubits}k{num_energy_bits}d{delta}.pkl', 'rb') as f:
    circ_state = pickle.load(f)

circ_sys_dm = partial_trace(circ_state, list(range(2 + num_energy_bits))).data
circ_sys_dm /= np.trace(circ_sys_dm)

In [20]:
dist = np.linalg.norm(dm_we_should_get - circ_sys_dm)
print(f'Basic distance = {dist:.4f}')
tr_dist = qt.tracedist(qt.Qobj(dm_we_should_get), qt.Qobj(circ_sys_dm))
print(f'Trace distance = {tr_dist:.4f}')
fidelity = qt.fidelity(qt.Qobj(dm_we_should_get), qt.Qobj(circ_sys_dm))
print(f'Fidelity = {fidelity:.4f}')

Basic distance = 0.0109
Trace distance = 0.0086
Fidelity = 0.9997


In [21]:
# diamond_norm = qt.dnorm(qt.Qobj(dm_we_should_get), qt.Qobj(circ_sys_dm))
# print(f'Diamond norm = {diamond_norm:.4f}')